In [47]:
# An example of building up the structure of a figure as a Python dictionary
# and then constructing a graph object figure from that dictionary.

In [48]:
# https://plotly.com/python/animations/#using-a-slider-and-buttons
# https://raw.githubusercontent.com/plotly/datasets/master/gapminderDataFiveYear.csv

In [49]:
from datetime import datetime

import pandas as pd
import plotly.graph_objects as go

import plotly.io as pio
pio.renderers.default = 'chrome' # https://community.plotly.com/t/plotly-doesnt-load-most-of-the-time/32095/3

In [50]:
dataset = pd.read_csv('time_series/rolling-time-series.csv')

In [51]:
dataset = dataset.replace(regex=['Australia'], value='Australia')
dataset = dataset.replace(regex=['India'], value='India')
dataset = dataset.replace(regex=['Italy'], value='Italia')
dataset = dataset.replace(regex=['Korea, South'], value='Corea del Sur')
dataset = dataset.replace(regex=['Mexico'], value='México')
dataset = dataset.replace(regex=['Switzerland'], value='Suiza')
dataset = dataset.replace(regex=['United Kingdom'], value='Reino Unido')
dataset = dataset.replace(regex=['US'], value='USA')

In [52]:
dataset['Date_Confirmed'] = dataset['Date_Confirmed'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))

In [53]:
import janitor
dataset = dataset.filter_date('Date_Confirmed','2020-03-10','2020-04-28')

In [54]:
# Make list of regions

regions = []

for region in dataset['Region']:
    if region not in regions:
        regions.append(region)

In [55]:
years = sorted(list(set(dataset['Date_Confirmed'])))

In [56]:
filled_df = []

for region in regions:
    for year in years:
        dataset_by_year = dataset[dataset['Date_Confirmed'] == year]
        dataset_by_year_and_region = dataset_by_year[dataset_by_year['Region'] == region]
        if len(dataset_by_year_and_region) == 0:
            dataset_by_year_less = dataset[dataset['Date_Confirmed'] < year]
            dataset_by_year_and_region_less = dataset_by_year_less[dataset_by_year_less['Region'] == region]
            if len(dataset_by_year_and_region_less) == 0:
                filled_df.append({'Date_Confirmed': year, 'Region': region, 'total':0, 'daily':0})
            else:
                last_dict = dataset_by_year_and_region_less.iloc[0].to_dict()
                last_dict['Date_Confirmed'] = year
                filled_df.append(last_dict)
        else:
            filled_df.append(dataset_by_year_and_region.iloc[0].to_dict())

In [57]:
dataset = pd.DataFrame(filled_df)

In [58]:
dataset['total'].max()

931371.0

In [59]:
dataset['daily'].max()

31595.0

In [60]:
dataset.tail()

,Date_Confirmed,Region,total,daily
395,2020-04-24,México,9869.0,857.0
396,2020-04-25,México,10775.0,906.0
397,2020-04-26,México,11692.0,917.0
398,2020-04-27,México,12657.0,965.0
399,2020-04-28,México,13693.0,1036.0


In [61]:
len(str(dataset['total'].max()))+1

9

In [62]:
fig_dict = {
  'data': [],
  'layout': {},
  'frames': []
}

In [63]:
fig_dict['layout']['title'] = {
  'text': 'Trayectoria global de COVID-19',
  'y': 0.9,
  'x': 0.5,
  'xanchor': 'center',
  'yanchor': 'top'
}
fig_dict['layout']['xaxis'] = {
  'title': 'Casos Totales',
  'type': 'log',
  'range': [0, 6]
}
fig_dict['layout']['yaxis'] = {
  'title': 'Casos Nuevos',
  'type': 'log',
  'range': [0, 6]
}
fig_dict['layout']['hovermode'] = 'closest'
fig_dict['layout']['sliders'] = {
  'args': [
    'transition', {
      'duration': 200,
      'easing': 'cubic-in-out'
    }
  ],
  'initialValue': datetime.strptime('10-03-2020', '%d-%m-%Y').isoformat().split('T')[0],
  'plotlycommand': 'animate',
  'values': years,
  'visible': True
}
fig_dict['layout']['updatemenus'] = [{
  'buttons': [{
      'args': [None, {
        'frame': {
          'duration': 500,
          'redraw': False
        },
        'fromcurrent': True,
        'transition': {
          'duration': 300,
          'easing': 'quadratic-in-out'
        }
      }],
      'label': 'Play',
      'method': 'animate'
    },
    {
      'args': [
        [None], {
          'frame': {
            'duration': 0,
            'redraw': False
          },
          'mode': 'immediate',
          'transition': {
            'duration': 0
          }
        }
      ],
      'label': 'Pause',
      'method': 'animate'
    }
  ],
  'direction': 'left',
  'pad': {
    'r': 10,
    't': 30,
  },
  'showactive': True,
  'type': 'buttons',
  'x': 0.1,
  'xanchor': 'right',
  'y': 0,
  'yanchor': 'top'
}]

In [64]:
sliders_dict = {
  'active': 0,
  'yanchor': 'top',
  'xanchor': 'left',
  'currentvalue': {
    'font': {
      'size': 15
    },
    'prefix': 'Fecha:',
    'visible': True,
    'xanchor': 'right'
  },
  'transition': {
    'duration': 300,
    'easing': 'cubic-in-out'
  },
  'pad': {
    'r': 0,
    't': 50,
  },
  'len': 1,
  'steps': []
}

In [65]:
year = datetime.strptime('2020-03-10', '%Y-%m-%d')

for region in regions:
    
    dataset_by_year = dataset[dataset['Date_Confirmed'] == year]
    dataset_by_year_and_region = dataset_by_year[dataset_by_year['Region'] == region]
    
    line_color = 'red' if region == 'México' else '#A9A9A9'
    
    data_dict = {
        'x': list(dataset_by_year_and_region['total']),
        'y': list(dataset_by_year_and_region['daily']),
        'mode': 'lines',
        'text': list(dataset_by_year_and_region['Region']),
        'line_color': line_color,
        #'hoverinfo': 'none',
    }
    
    data_dict_2 = {
        'x': list(dataset_by_year_and_region['total']),
        'y': list(dataset_by_year_and_region['daily']),
        'mode': 'lines',
        'text': list(dataset_by_year_and_region['Region']),
        'line_color': line_color,
        #'hoverinfo': 'none',
    }
    
    fig_dict['data'].append(data_dict)
    fig_dict['data'].append(data_dict_2)

In [66]:
# Make frames

In [67]:
for y, year in enumerate(years):
    
    frame = {'data': [], 'name': str(year.isoformat().split('T')[0])}
    frame_2 = frame.copy()
    
    dataset_by_year = dataset[dataset['Date_Confirmed'] <= year]
    
    for region in regions:
        dataset_by_year_and_region = dataset_by_year[
            dataset_by_year['Region'] == region]
        
        line_color = 'red' if region == 'México' else '#A9A9A9'
            
        data_dict = {
                    'x': list(dataset_by_year_and_region['total']),
                    'y': list(dataset_by_year_and_region['daily']),
                    'mode': 'lines',
                    'text': list(dataset_by_year_and_region['Region']),
                    'textposition': 'bottom center',
                    'name': region,
                    'line_color': line_color,
                    #'hoverinfo': 'none',
            }
        
        data_dict_2 = {
                    'x': [list(dataset_by_year_and_region['total'])[-1]],
                    'y': [list(dataset_by_year_and_region['daily'])[-1]],
                    'mode': 'markers+text',
                    'text': [list(dataset_by_year_and_region['Region'])[-1]],
                    'textposition': 'bottom center',
                    'name': region,
                    'line_color': line_color,
                    #'hoverinfo': 'none',
            }
        
        frame['data'].append(data_dict)
        frame['data'].append(data_dict_2)
        
    fig_dict['frames'].append(frame)
    
    slider_step = {'args': [
        [year.isoformat().split('T')[0]],
        {'frame': {'duration': 200, 'redraw': True},
         'mode': 'immediate',
         'transition': {'duration': 200}}
    ],
        'label': year.isoformat().split('T')[0],
        'method': 'animate'}
    
    sliders_dict['steps'].append(slider_step)
    
fig_dict['layout']['sliders'] = [sliders_dict]

In [68]:
fig = go.Figure(fig_dict)

fig.show()

In [69]:
# 'hovertemplate': 'Nuevos casos: %{y:.0f}</br>Casos Totales: %{x:.0f}<extra></extra>'

In [70]:
# fig_dict['frames'][-1]

In [71]:
# https://plotly.com/python/interactive-html-export/
fig.write_html('plot.html')